# 0. 라이브러리

In [ ]:
!pip install konlpy
!pip install networkx
!pip install apyori
!pip install pyLDAvis==2.1.2
!pip install wordcloud
!pip install twitter
!pip install gensim
!pip install openpyxl
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 7.1 MB/s 
     |████████████████████████████████| 453 kB 59.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=4de3cd712faf4d7aabe3703e0db0d2b89aeb787987d3026865282cb61b01fb2c
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.8 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=47b468191dea3f7da604483c6bada08d77b3a0b4b608a53c2dfd2d20

In [ ]:
# 기본 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib as mpl
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import re
import pickle 
import openpyxl
from IPython.display import set_matplotlib_formats 

# 불용어
import nltk
nltk.download('stopwords')

# konlpy
from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Twitter
twitter = Twitter()

# counter
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
from sklearn.feature_extraction.text import TfidfVectorizer

# wordcloud
from wordcloud import WordCloud 

# lda
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import pyLDAvis.gensim
import pyLDAvis

# sna
import networkx as nx
from apyori import apriori

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
# 운영체제별 한글 폰트 설정
if os.name == 'posix': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif os.name == 'nt': # Windows 환경 폰트 설정
    plt.rc('font', family='NanumGothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

NameError: ignored

In [ ]:
# 이 셀까지 실행 후 런타임 초기화 요망
# 런타임 초기화 후 처음부터 재시작, 이 셀은 실행하지 않음
# 이 셀 스킵 후 다음 셀부터 시작
!sudo apt-get install -y fonts-NanumGothic
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!apt -qq -y install fonts-nanum
import matplotlib.font_manager as fm
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf' 
font = fm.FontProperties(fname=path, size=9)
plt.rc('font', family='NanumGothic')
mpl.font_manager._rebuild()

# 1. 불용어 리스트

In [ ]:
korean_stopwords_path = "/korean_stopwords.txt" # 기본 불용어 파일 

with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# 불용어 추가
other_stopwords = ['추가','불용어','리스트']
for stopword in other_stopwords:
    stopwords.append(stopword)

# 2. 워드 클라우드

## 2-1. 토큰화/불용어제거

In [ ]:
def text_preprocessing(x):
  title_corpus = x['title'].tolist()  # 컬럼명 title

  # 형태소 분석
  morphs = []
  for sentence in title_corpus:
    morphs.append(twitter.pos(str(sentence)))
  print(morphs)
  print('----------------------------------------------------------------------------------------------------')
  print()
  print()

  # 품사 지정(명사)
  noun_adj_adv_list = []
  for sentence in morphs : 
    for word, tag in sentence :
        if tag in ['Noun'] and ("것" not in word) and ("내" not in word)and ("나" not in word)and ("수"not in word) and("게"not in word)and("말"not in word): 
            noun_adj_adv_list.append(word) 

  print(noun_adj_adv_list)
  print('----------------------------------------------------------------------------------------------------')
  print()
  print()

  # 불용어 제거
  step1 = [x for x in noun_adj_adv_list if not x in stopwords] # N 입력 후 불용어 제거
  step2 = re.sub(r'[^ A-Za-z0-9가-힣+]', '', str(step1)) # N 입력 후 정규화
  shortword = re.compile(r'\W*\b\w{1}\b')
  step3 = shortword.sub('', step2) # N 입력 후 한글자 제거
  
  return step3 # 데이터에 따라 괜찮은 전처리 방식 return 


## 2-2. 워드클라우드 함수

In [ ]:
def wordcloud(x):
  # counter
  count = Counter(x)
  words = dict(count.most_common())
  print(words)
  print('----------------------------------------------------------------------------------------------------')
  print()
  print()

  # 워드클라우드
  wordcloud = WordCloud(font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf', 
                          background_color='white',colormap = "Accent_r", 
                          width=1500, height=1000).generate_from_frequencies(words) 
  plt.imshow(wordcloud) 
  plt.axis('off') 
  plt.show()
  print('----------------------------------------------------------------------------------------------------')
  print()
  print()


# 3. LDA

## 3-1. LDA용 전처리 함수

In [ ]:
def preprocessing(x):
    title_corpus =" ".join(map(str,x['title']))

    df_noun = okt.nouns(title_corpus)

    df_noun_x = [x for x in df_noun if not x in stopwords] # N 입력 후 불용어 제거

    df_N_re = re.sub(r'[^ A-Za-z0-9가-힣+]', '', str(df_noun_x)) # N 입력 후 정규화

    shortword = re.compile(r'\W*\b\w{1}\b')
    result = shortword.sub('', df_N_re) # N 입력 후 한글자 제거

    return result

In [ ]:
# 한글만
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 
    result = hangul.sub('', str(text))
    return result

In [ ]:
# 한글&영어
def text_cleaning(text):
    hangul = re.compile('[^A-Za-z0-9가-힣]') 
    result = hangul.sub('', str(text))
    return result

In [ ]:
def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    # 한글자 키워드 제거
    nouns = [noun for noun in nouns if len(noun) > 1]
    # 불용어 제거
    nouns = [noun for noun in nouns if noun not in stopwords]
    return nouns

## 3-2. LDA

In [ ]:
tfidfv = TfidfVectorizer().fit([result])
print(tfidfv.transform([result]).toarray())
qt_tfidf = tfidfv.vocabulary_
print(qt_tfidf)

In [ ]:
id2word = corpora.Dictionary([result.split()]) # 정수인코딩과 빈도수 생성
corpus = [id2word.doc2bow(text) for text in [result.split()]] # 출현빈도가 작거나 자주 등장하는 단어는 제거
print(corpus)
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=id2word, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)
    pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, id2word)
#vis = pyLDAvis.gensim.prepare(ldamodel, corpus_tfidf, id2word)
pyLDAvis.display(vis)

# 4. SNA

In [ ]:
jp['ko_text'] = jp['title'].apply(lambda x : text_cleaning(x))
jp['nouns'] = jp['ko_text'].apply(lambda x: get_nouns(x))
print(jp)
transactions = jp['nouns'].tolist()
transactions = [transaction for transaction in transactions if transaction] # 공백 문자열 방지
print(transactions)
# 연관 분석 수행
results = list(apriori(transactions,
    min_support=0.06,
    min_confidence=0.05,
    min_lift=1.0,
    max_length=2))
print(results)
columns = ['source', 'target', 'support']
network_df = pd.DataFrame(columns=columns)

# 규칙의 조건절을 source, 결과절을 target, 지지도를 support 라는 데이터 프레임의 피처로 변환
for result in results:
    if len(result.items) == 2:
        items = [x for x in result.items]
        row = [items[0], items[1], result.support]
        series = pd.Series(row, index=network_df.columns)
        network_df = network_df.append(series, ignore_index=True)

network_df.head()

In [ ]:
corpus = "".join(jp['ko_text'].tolist())
print(corpus)
# 명사 키워드 추출
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(corpus)
# 불용어 제거 추가
df_noun_x = [x for x in nouns if not x in stopwords]
count = Counter(df_noun_x)
# 한글자 키워드 제거
remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})

# 키워드와 키워드 빈도 점수를 ‘node’, ‘nodesize’ 라는 데이터 프레임의 피처로 생성
node_df = pd.DataFrame(remove_char_counter.items(), columns=['node', 'nodesize'])
node_df = node_df[node_df['nodesize'] >= 10] # 시각화의 편의를 위해 ‘nodesize’ 5 이하 제거
node_df.head()
plt.figure(figsize=(25,25))

# networkx 그래프 객체 생성
G = nx.Graph()

# node_df의 키워드 빈도수를 데이터로 하여, 네트워크 그래프의 ‘노드’ 역할을 하는 원 생성
for index, row in node_df.iterrows():
    G.add_node(row['node'], nodesize=row['nodesize'])

# network_df의 연관 분석 데이터를 기반으로, 네트워크 그래프의 ‘관계’ 역할을 하는 선 생성
for index, row in network_df.iterrows():
    G.add_weighted_edges_from([(row['source'], row['target'], row['support'])])

# 그래프 디자인과 관련된 파라미터 설정.
pos = nx.spring_layout(G, k=0.6, iterations=50)
sizes = [G.nodes[node]['nodesize']*25 for node in G]
nx.draw(G, pos=pos, node_size=sizes)

nx.draw_networkx_labels(G, pos=pos, font_family="NanumGothic", font_size=25)



# 그래프 출력
ax = plt.gca()
plt.show()